## Skimage包

In [ ]:
import os
from skimage import io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

In [ ]:
def SLIC_SkImage(img_path, n_segments, compactness, max_iter):
    image = io.imread(img_path)
    segments = slic(image, n_segments=n_segments, compactness=compactness, max_iter=max_iter)
    return mark_boundaries(image, segments)

def ImgSeg_SkImage(dir_path, n_segments=100, compactness=10.0, max_iter=10):
    for _, _, files in os.walk(dir_path):
        files.sort()#
        for filename in files:
            if (filename != ".DS_Store"):#
                img_path = dir_path + filename
                result = SLIC_SkImage(img_path, n_segments, compactness, max_iter)
                plt.imshow(result)
                save_path = "resultfigs/" + filename[:-4] + "_seg%d_comp%.2f.jpg"%(n_segments,compactness)
                plt.savefig(save_path)

In [ ]:
ImgSeg_SkImage("testfigs/pos/", n_segments=30)
ImgSeg_SkImage("testfigs/pos/", n_segments=40)
ImgSeg_SkImage("testfigs/pos/", n_segments=50)
ImgSeg_SkImage("testfigs/pos/", n_segments=40, compactness=15)
ImgSeg_SkImage("testfigs/pos/", n_segments=40, compactness=20)

## OpenCV包

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
def SLIC_OpenCV(img_name, algorithm, region_size, ruler, max_iter):
    image = cv2.imread(img_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    slic = cv2.ximgproc.createSuperpixelSLIC(image, algorithm = algorithm, region_size = region_size, ruler = ruler) 
    slic.iterate(max_iter)
    
    mask_slic = slic.getLabelContourMask()#获取Mask，超像素边缘Mask==1
    mask_inv_slic = cv2.bitwise_not(mask_slic)  
    label_slic = slic.getLabels()#获取超像素标签
    
    img_slic = cv2.bitwise_or(src1=image, src2=image, mask=mask_inv_slic) #在原图上绘制超像素边界
    
    return img_slic

"""
cv::ximgproc::SLIC = 100,
cv::ximgproc::SLICO = 101,
cv::ximgproc::MSLIC = 102 
"""

def ImgSeg_OpenCV(src_dir, dst_dir, algorithm=101, region_size=20, ruler=20.0, max_iter=10):
    for _, _, files in os.walk(src_dir):
        files.sort()#
        for filename in files:
            if (filename != ".DS_Store"):#
                img_name = src_dir + filename
                result = SLIC_OpenCV(img_name, algorithm, region_size, ruler, max_iter)
                result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB) 
                save_path = dst_dir + filename[:-4] + "_algo%d_regsize%d_ruler%.1f.jpg"%(algorithm,region_size,ruler)
                cv2.imwrite(save_path,result)

In [ ]:
ImgSeg_OpenCV(path, algorithm = 100, region_size=30, ruler=40)#SLIC
ImgSeg_OpenCV(path, algorithm = 100, region_size=40, ruler=40)#SLIC
ImgSeg_OpenCV(path, algorithm = 100, region_size=50, ruler=40)#SLIC
ImgSeg_OpenCV(path, algorithm = 100, region_size=30, ruler=50)#SLIC
ImgSeg_OpenCV(path, algorithm = 100, region_size=40, ruler=50)#SLIC
ImgSeg_OpenCV(path, algorithm = 100, region_size=50, ruler=50)#SLIC

In [ ]:
ImgSeg_OpenCV(path, algorithm = 102, region_size=40, ruler=40)#MSLIC
ImgSeg_OpenCV(path, algorithm = 102, region_size=50, ruler=40)#MSLIC
ImgSeg_OpenCV(path, algorithm = 102, region_size=30, ruler=50)#MSLIC
ImgSeg_OpenCV(path, algorithm = 102, region_size=40, ruler=50)#MSLIC